In [ ]:
import cv2
import mediapipe as mp
import math
import numpy as np
from enum import Enum
from typing import List, Tuple, Optional

class FingerState(Enum):
    """手指狀態枚舉"""
    EXTENDED = 0  # 伸直
    BENT = 1      # 彎曲

class HandGesture(Enum):
    """手勢類型枚舉"""
    GOOD = "good"
    BAD = "bad" 
    NO = "no!!!"
    ROCK = "ROCK!"
    ZERO = "0"
    ONE = "1"
    TWO = "2"
    THREE = "3"
    FOUR = "4"
    FIVE = "5"
    SIX = "6"
    SEVEN = "7"
    EIGHT = "8"
    NINE = "9"
    OK = "ok"
    PINK = "pink"
    UNKNOWN = ""

class Config:
    """設定常數"""
    # 影像設定
    FRAME_WIDTH = 540
    FRAME_HEIGHT = 310
    
    # 手指角度閾值
    FINGER_ANGLE_THRESHOLD = 50
    
    # MediaPipe 設定
    MODEL_COMPLEXITY = 0
    MIN_DETECTION_CONFIDENCE = 0.5
    MIN_TRACKING_CONFIDENCE = 0.5
    
    # 顯示設定
    FONT_FACE = cv2.FONT_HERSHEY_SIMPLEX
    LINE_TYPE = cv2.LINE_AA
    TEXT_COLOR = (255, 255, 255)
    TEXT_THICKNESS = 10
    TEXT_SCALE = 5
    TEXT_POSITION = (30, 120)
    
    # 馬賽克設定
    MOSAIC_SIZE = 8
    MOSAIC_PADDING = 10

class AngleCalculator:
    """角度計算器"""
    
    @staticmethod
    def vector_2d_angle(v1: Tuple[float, float], v2: Tuple[float, float]) -> float:
        """計算兩個二維向量之間的角度"""
        v1_x, v1_y = v1
        v2_x, v2_y = v2
        
        try:
            dot_product = v1_x * v2_x + v1_y * v2_y
            magnitude_v1 = math.sqrt(v1_x**2 + v1_y**2)
            magnitude_v2 = math.sqrt(v2_x**2 + v2_y**2)
            cos_angle = dot_product / (magnitude_v1 * magnitude_v2)
            angle = math.degrees(math.acos(cos_angle))
        except (ZeroDivisionError, ValueError):
            angle = 180
        
        return angle
    
    @staticmethod
    def calculate_finger_angles(landmarks: List[Tuple[float, float]]) -> List[float]:
        """計算五個手指的角度"""
        angles = []
        
        # 定義手指節點索引
        finger_indices = [
            (0, 2, 3, 4),   # 大拇指
            (0, 6, 7, 8),   # 食指
            (0, 10, 11, 12), # 中指
            (0, 14, 15, 16), # 無名指
            (0, 18, 19, 20)  # 小拇指
        ]
        
        for base_idx, joint1_idx, joint2_idx, tip_idx in finger_indices:
            v1 = (
                int(landmarks[base_idx][0]) - int(landmarks[joint1_idx][0]),
                int(landmarks[base_idx][1]) - int(landmarks[joint1_idx][1])
            )
            v2 = (
                int(landmarks[joint2_idx][0]) - int(landmarks[tip_idx][0]),
                int(landmarks[joint2_idx][1]) - int(landmarks[tip_idx][1])
            )
            
            angle = AngleCalculator.vector_2d_angle(v1, v2)
            angles.append(angle)
        
        return angles

class GestureRecognizer:
    """手勢識別器"""
    
    def __init__(self):
        self.gesture_patterns = {
            # (大拇指, 食指, 中指, 無名指, 小拇指) - True表示伸直, False表示彎曲
            HandGesture.GOOD: (True, False, False, False, False),
            HandGesture.BAD: (False, True, True, True, True),
            HandGesture.NO: (False, False, True, False, False),
            HandGesture.ROCK: (True, False, False, False, True),
            HandGesture.ZERO: (False, False, False, False, False),
            HandGesture.ONE: (False, True, False, False, False),
            HandGesture.TWO: (False, True, True, False, False),
            HandGesture.THREE: (False, True, True, True, False),
            HandGesture.FOUR: (False, True, True, True, True),
            HandGesture.FIVE: (True, True, True, True, True),
            HandGesture.SIX: (True, False, False, False, True),
            HandGesture.SEVEN: (True, True, False, False, False),
            HandGesture.EIGHT: (True, True, True, False, False),
            HandGesture.NINE: (True, True, True, True, False),
            HandGesture.OK: (False, False, True, True, True),
            HandGesture.PINK: (False, False, False, False, True),
        }
    
    def _angles_to_finger_states(self, angles: List[float]) -> Tuple[bool, ...]:
        """將角度轉換為手指狀態"""
        return tuple(angle < Config.FINGER_ANGLE_THRESHOLD for angle in angles)
    
    def recognize(self, finger_angles: List[float]) -> HandGesture:
        """識別手勢"""
        finger_states = self._angles_to_finger_states(finger_angles)
        
        for gesture, pattern in self.gesture_patterns.items():
            if finger_states == pattern:
                return gesture
        
        return HandGesture.UNKNOWN

class EffectProcessor:
    """效果處理器"""
    
    @staticmethod
    def apply_mosaic(image: np.ndarray, x_min: int, y_min: int, 
                    x_max: int, y_max: int) -> np.ndarray:
        """應用馬賽克效果"""
        if x_min >= x_max or y_min >= y_max:
            return image
        
        # 提取區域
        region = image[y_min:y_max, x_min:x_max]
        if region.size == 0:
            return image
        
        # 應用馬賽克
        height, width = region.shape[:2]
        mosaic = cv2.resize(region, (Config.MOSAIC_SIZE, Config.MOSAIC_SIZE), 
                          interpolation=cv2.INTER_LINEAR)
        mosaic = cv2.resize(mosaic, (width, height), 
                          interpolation=cv2.INTER_NEAREST)
        
        # 將馬賽克區域貼回原圖
        image[y_min:y_max, x_min:x_max] = mosaic
        return image
    
    @staticmethod
    def draw_text(image: np.ndarray, text: str) -> np.ndarray:
        """在圖像上繪製文字"""
        cv2.putText(image, text, Config.TEXT_POSITION, Config.FONT_FACE,
                   Config.TEXT_SCALE, Config.TEXT_COLOR, Config.TEXT_THICKNESS,
                   Config.LINE_TYPE)
        return image

class HandTracker:
    """手部追蹤器"""
    
    def __init__(self):
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            model_complexity=Config.MODEL_COMPLEXITY,
            min_detection_confidence=Config.MIN_DETECTION_CONFIDENCE,
            min_tracking_confidence=Config.MIN_TRACKING_CONFIDENCE
        )
        self.angle_calculator = AngleCalculator()
        self.gesture_recognizer = GestureRecognizer()
        self.effect_processor = EffectProcessor()
    
    def extract_landmarks(self, hand_landmarks) -> Tuple[List[Tuple[float, float]], List[int], List[int]]:
        """提取手部特徵點座標"""
        landmarks = []
        x_coords = []
        y_coords = []
        
        for landmark in hand_landmarks.landmark:
            x = landmark.x * Config.FRAME_WIDTH
            y = landmark.y * Config.FRAME_HEIGHT
            landmarks.append((x, y))
            x_coords.append(int(x))
            y_coords.append(int(y))
        
        return landmarks, x_coords, y_coords
    
    def get_bounding_box(self, x_coords: List[int], y_coords: List[int]) -> Tuple[int, int, int, int]:
        """獲取手部邊界框"""
        x_min = max(0, min(x_coords) - Config.MOSAIC_PADDING)
        y_min = max(0, min(y_coords) - Config.MOSAIC_PADDING)
        x_max = min(Config.FRAME_WIDTH, max(x_coords))
        y_max = min(Config.FRAME_HEIGHT, max(y_coords))
        
        return x_min, y_min, x_max, y_max
    
    def process_frame(self, frame: np.ndarray) -> np.ndarray:
        """處理單一幀圖像"""
        # 轉換顏色空間
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(rgb_frame)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # 提取特徵點
                landmarks, x_coords, y_coords = self.extract_landmarks(hand_landmarks)
                
                if landmarks:
                    # 計算手指角度
                    finger_angles = self.angle_calculator.calculate_finger_angles(landmarks)
                    
                    # 識別手勢
                    gesture = self.gesture_recognizer.recognize(finger_angles)
                    
                    # 根據手勢應用效果
                    if gesture == HandGesture.NO:
                        x_min, y_min, x_max, y_max = self.get_bounding_box(x_coords, y_coords)
                        frame = self.effect_processor.apply_mosaic(frame, x_min, y_min, x_max, y_max)
                    elif gesture != HandGesture.UNKNOWN:
                        frame = self.effect_processor.draw_text(frame, gesture.value)
        
        return frame
    
    def __del__(self):
        """清理資源"""
        if hasattr(self, 'hands'):
            self.hands.close()

class HandGestureApp:
    """手勢識別應用主類"""
    
    def __init__(self):
        self.cap = cv2.VideoCapture(0)
        self.hand_tracker = HandTracker()
        
        if not self.cap.isOpened():
            raise RuntimeError("無法開啟攝影機")
    
    def run(self):
        """運行應用程式"""
        print("手勢識別應用已啟動，按 'q' 鍵退出")
        
        try:
            while True:
                ret, frame = self.cap.read()
                if not ret:
                    print("無法讀取攝影機畫面")
                    break
                
                # 調整畫面大小
                frame = cv2.resize(frame, (Config.FRAME_WIDTH, Config.FRAME_HEIGHT))
                
                # 處理手勢識別
                processed_frame = self.hand_tracker.process_frame(frame)
                
                # 顯示結果
                cv2.imshow('Hand Gesture Recognition', processed_frame)
                
                # 檢查退出條件
                if cv2.waitKey(5) & 0xFF == ord('q'):
                    break
                    
        except KeyboardInterrupt:
            print("程式被中斷")
        finally:
            self.cleanup()
    
    def cleanup(self):
        """清理資源"""
        self.cap.release()
        cv2.destroyAllWindows()

# 運行應用程式
if __name__ == "__main__":
    app = HandGestureApp()
    app.run()


In [9]:
import cv2
import mediapipe as mp
import math
import numpy as np
from enum import Enum
from typing import List, Tuple, Optional
import random

class FingerState(Enum):
    """手指狀態枚舉"""
    EXTENDED = 0  # 伸直
    BENT = 1      # 彎曲

class HandGesture(Enum):
    """手勢類型枚舉"""
    GOOD = "good"
    BAD = "bad" 
    NO = "no!!!"
    ROCK = "ROCK!"
    ZERO = "0"
    ONE = "1"
    TWO = "2"
    THREE = "3"
    FOUR = "4"
    FIVE = "5"
    SIX = "6"
    SEVEN = "7"
    EIGHT = "8"
    NINE = "9"
    OK = "ok"
    PINK = "pink"
    HEART = "heart"  # 新增愛心手勢
    UNKNOWN = ""

class Config:
    """設定常數"""
    # 影像設定
    FRAME_WIDTH = 540
    FRAME_HEIGHT = 310
    
    # 手指角度閾值
    FINGER_ANGLE_THRESHOLD = 50
    
    # MediaPipe 設定
    MODEL_COMPLEXITY = 0
    MIN_DETECTION_CONFIDENCE = 0.5
    MIN_TRACKING_CONFIDENCE = 0.5
    
    # 顯示設定
    FONT_FACE = cv2.FONT_HERSHEY_SIMPLEX
    LINE_TYPE = cv2.LINE_AA
    TEXT_COLOR = (255, 255, 255)
    TEXT_THICKNESS = 10
    TEXT_SCALE = 5
    TEXT_POSITION = (30, 120)
    
    # 馬賽克設定
    MOSAIC_SIZE = 8
    MOSAIC_PADDING = 10
    
    # 愛心手勢設定
    HEART_DISTANCE_THRESHOLD = 100  # 雙手距離閾值
    HEART_ANGLE_THRESHOLD = 30      # 角度閾值
    PINK_FILTER_INTENSITY = 0.3     # 粉色濾鏡強度

class AngleCalculator:
    """角度計算器"""
    
    @staticmethod
    def vector_2d_angle(v1: Tuple[float, float], v2: Tuple[float, float]) -> float:
        """計算兩個二維向量之間的角度"""
        v1_x, v1_y = v1
        v2_x, v2_y = v2
        
        try:
            dot_product = v1_x * v2_x + v1_y * v2_y
            magnitude_v1 = math.sqrt(v1_x**2 + v1_y**2)
            magnitude_v2 = math.sqrt(v2_x**2 + v2_y**2)
            cos_angle = dot_product / (magnitude_v1 * magnitude_v2)
            angle = math.degrees(math.acos(cos_angle))
        except (ZeroDivisionError, ValueError):
            angle = 180
        
        return angle
    
    @staticmethod
    def calculate_finger_angles(landmarks: List[Tuple[float, float]]) -> List[float]:
        """計算五個手指的角度"""
        angles = []
        
        # 定義手指節點索引
        finger_indices = [
            (0, 2, 3, 4),   # 大拇指
            (0, 6, 7, 8),   # 食指
            (0, 10, 11, 12), # 中指
            (0, 14, 15, 16), # 無名指
            (0, 18, 19, 20)  # 小拇指
        ]
        
        for base_idx, joint1_idx, joint2_idx, tip_idx in finger_indices:
            v1 = (
                int(landmarks[base_idx][0]) - int(landmarks[joint1_idx][0]),
                int(landmarks[base_idx][1]) - int(landmarks[joint1_idx][1])
            )
            v2 = (
                int(landmarks[joint2_idx][0]) - int(landmarks[tip_idx][0]),
                int(landmarks[joint2_idx][1]) - int(landmarks[tip_idx][1])
            )
            
            angle = AngleCalculator.vector_2d_angle(v1, v2)
            angles.append(angle)
        
        return angles

class HeartGestureDetector:
    """愛心手勢檢測器"""
    
    @staticmethod
    def detect_heart_gesture(left_landmarks: Optional[List[Tuple[float, float]]], 
                           right_landmarks: Optional[List[Tuple[float, float]]]) -> bool:
        """檢測雙手是否組成愛心手勢"""
        if not left_landmarks or not right_landmarks:
            return False
        
        # 獲取大拇指指尖和食指指尖位置
        left_thumb_tip = left_landmarks[4]   # 左手大拇指指尖
        left_index_tip = left_landmarks[8]   # 左手食指指尖
        right_thumb_tip = right_landmarks[4] # 右手大拇指指尖
        right_index_tip = right_landmarks[8] # 右手食指指尖
        
        # 計算雙手距離
        thumb_distance = math.sqrt(
            (left_thumb_tip[0] - right_thumb_tip[0])**2 + 
            (left_thumb_tip[1] - right_thumb_tip[1])**2
        )
        
        index_distance = math.sqrt(
            (left_index_tip[0] - right_index_tip[0])**2 + 
            (left_index_tip[1] - right_index_tip[1])**2
        )
        
        # 檢查距離是否合理（形成愛心的大小）
        if thumb_distance > Config.HEART_DISTANCE_THRESHOLD or index_distance > Config.HEART_DISTANCE_THRESHOLD:
            return False
        
        # 檢查手指角度（大拇指和食指應該彎曲成特定角度）
        left_angles = AngleCalculator.calculate_finger_angles(left_landmarks)
        right_angles = AngleCalculator.calculate_finger_angles(right_landmarks)
        
        # 愛心手勢：大拇指和食指應該彎曲，其他手指收起
        left_heart_pattern = (
            left_angles[0] > Config.FINGER_ANGLE_THRESHOLD,  # 大拇指彎曲
            left_angles[1] > Config.FINGER_ANGLE_THRESHOLD,  # 食指彎曲
            left_angles[2] < Config.FINGER_ANGLE_THRESHOLD,  # 中指收起
            left_angles[3] < Config.FINGER_ANGLE_THRESHOLD,  # 無名指收起
            left_angles[4] < Config.FINGER_ANGLE_THRESHOLD   # 小拇指收起
        )
        
        right_heart_pattern = (
            right_angles[0] > Config.FINGER_ANGLE_THRESHOLD,  # 大拇指彎曲
            right_angles[1] > Config.FINGER_ANGLE_THRESHOLD,  # 食指彎曲
            right_angles[2] < Config.FINGER_ANGLE_THRESHOLD,  # 中指收起
            right_angles[3] < Config.FINGER_ANGLE_THRESHOLD,  # 無名指收起
            right_angles[4] < Config.FINGER_ANGLE_THRESHOLD   # 小拇指收起
        )
        
        return all(left_heart_pattern) and all(right_heart_pattern)

class GestureRecognizer:
    """手勢識別器"""
    
    def __init__(self):
        self.gesture_patterns = {
            # (大拇指, 食指, 中指, 無名指, 小拇指) - True表示伸直, False表示彎曲
            HandGesture.GOOD: (True, False, False, False, False),
            HandGesture.NO: (False, False, True, False, False),
            HandGesture.ROCK: (True, False, False, False, True),
            HandGesture.ZERO: (False, False, False, False, False),
            HandGesture.ONE: (False, True, False, False, False),
            HandGesture.TWO: (False, True, True, False, False),
            HandGesture.THREE: (False, True, True, True, False),
            HandGesture.FOUR: (False, True, True, True, True),
            HandGesture.FIVE: (True, True, True, True, True),
            HandGesture.SIX: (True, False, False, False, True),
            HandGesture.SEVEN: (True, True, False, False, False),
            HandGesture.EIGHT: (True, True, True, False, False),
            HandGesture.NINE: (True, True, True, True, False),
            HandGesture.OK: (False, False, True, True, True),
            HandGesture.PINK: (False, False, False, False, True),
        }
        self.heart_detector = HeartGestureDetector()
    
    def _angles_to_finger_states(self, angles: List[float]) -> Tuple[bool, ...]:
        """將角度轉換為手指狀態"""
        return tuple(angle < Config.FINGER_ANGLE_THRESHOLD for angle in angles)
    
    def recognize(self, finger_angles: List[float]) -> HandGesture:
        """識別單手手勢"""
        finger_states = self._angles_to_finger_states(finger_angles)
        
        for gesture, pattern in self.gesture_patterns.items():
            if finger_states == pattern:
                return gesture
        
        return HandGesture.UNKNOWN
    
    def recognize_dual_hand(self, left_landmarks: Optional[List[Tuple[float, float]]], 
                          right_landmarks: Optional[List[Tuple[float, float]]]) -> HandGesture:
        """識別雙手手勢"""
        if self.heart_detector.detect_heart_gesture(left_landmarks, right_landmarks):
            return HandGesture.HEART
        return HandGesture.UNKNOWN

class EffectProcessor:
    """效果處理器"""
    
    def __init__(self):
        self.heart_particles = []
        self.heart_effect_timer = 0
    
    @staticmethod
    def apply_mosaic(image: np.ndarray, x_min: int, y_min: int, 
                    x_max: int, y_max: int) -> np.ndarray:
        """應用馬賽克效果"""
        if x_min >= x_max or y_min >= y_max:
            return image
        
        # 提取區域
        region = image[y_min:y_max, x_min:x_max]
        if region.size == 0:
            return image
        
        # 應用馬賽克
        height, width = region.shape[:2]
        mosaic = cv2.resize(region, (Config.MOSAIC_SIZE, Config.MOSAIC_SIZE), 
                          interpolation=cv2.INTER_LINEAR)
        mosaic = cv2.resize(mosaic, (width, height), 
                          interpolation=cv2.INTER_NEAREST)
        
        # 將馬賽克區域貼回原圖
        image[y_min:y_max, x_min:x_max] = mosaic
        return image
    
    def apply_pink_filter(self, image: np.ndarray) -> np.ndarray:
        """應用粉色濾鏡"""
        # 創建粉色遮罩
        pink_overlay = np.zeros_like(image, dtype=np.uint8)
        pink_overlay[:, :] = [180, 105, 255]  # BGR格式的粉色
        
        # 混合原圖和粉色遮罩
        filtered_image = cv2.addWeighted(image, 1 - Config.PINK_FILTER_INTENSITY, 
                                       pink_overlay, Config.PINK_FILTER_INTENSITY, 0)
        return filtered_image
    
    def draw_heart_shape(self, image: np.ndarray, center_x: int, center_y: int, 
                        size: int = 30, color: Tuple[int, int, int] = (255, 105, 180)) -> np.ndarray:
        """繪製愛心形狀"""
        # 使用數學函數繪製愛心
        heart_points = []
        for i in range(360):
            t = math.radians(i)
            # 愛心的參數方程
            x = 16 * math.sin(t)**3
            y = 13 * math.cos(t) - 5 * math.cos(2*t) - 2 * math.cos(3*t) - math.cos(4*t)
            
            # 縮放和平移
            heart_x = int(center_x + x * size / 16)
            heart_y = int(center_y - y * size / 16)
            
            if 0 <= heart_x < Config.FRAME_WIDTH and 0 <= heart_y < Config.FRAME_HEIGHT:
                heart_points.append((heart_x, heart_y))
        
        # 填充愛心
        if heart_points:
            heart_points = np.array(heart_points, np.int32)
            cv2.fillPoly(image, [heart_points], color)
        
        return image
    
    def update_heart_particles(self):
        """更新愛心粒子效果"""
        # 移除過期的粒子
        self.heart_particles = [p for p in self.heart_particles if p['life'] > 0]
        
        # 更新現有粒子
        for particle in self.heart_particles:
            particle['x'] += particle['vx']
            particle['y'] += particle['vy']
            particle['life'] -= 1
            particle['size'] = max(1, particle['size'] - 0.2)
    
    def add_heart_particles(self, center_x: int, center_y: int):
        """添加愛心粒子"""
        for _ in range(5):  # 每次添加5個粒子
            particle = {
                'x': center_x + random.randint(-50, 50),
                'y': center_y + random.randint(-50, 50),
                'vx': random.uniform(-2, 2),
                'vy': random.uniform(-3, -1),  # 向上飄
                'life': random.randint(30, 60),
                'size': random.randint(10, 20),
                'color': (255, random.randint(100, 255), random.randint(150, 255))
            }
            self.heart_particles.append(particle)
    
    def draw_heart_particles(self, image: np.ndarray) -> np.ndarray:
        """繪製愛心粒子效果"""
        for particle in self.heart_particles:
            if particle['life'] > 0:
                center = (int(particle['x']), int(particle['y']))
                size = int(particle['size'])
                if 0 <= center[0] < Config.FRAME_WIDTH and 0 <= center[1] < Config.FRAME_HEIGHT:
                    self.draw_heart_shape(image, center[0], center[1], 
                                        size//2, particle['color'])
        return image
    
    def apply_heart_effect(self, image: np.ndarray, center_x: int, center_y: int) -> np.ndarray:
        """應用愛心特效"""
        # 應用粉色濾鏡
        image = self.apply_pink_filter(image)
        
        # 繪製主要愛心
        image = self.draw_heart_shape(image, center_x, center_y, 40, (255, 20, 147))
        
        # 更新和繪製粒子效果
        self.update_heart_particles()
        
        # 定期添加新粒子
        self.heart_effect_timer += 1
        if self.heart_effect_timer % 10 == 0:  # 每10幀添加一次
            self.add_heart_particles(center_x, center_y)
        
        image = self.draw_heart_particles(image)
        
        return image
    
    @staticmethod
    def draw_text(image: np.ndarray, text: str) -> np.ndarray:
        """在圖像上繪製文字"""
        cv2.putText(image, text, Config.TEXT_POSITION, Config.FONT_FACE,
                   Config.TEXT_SCALE, Config.TEXT_COLOR, Config.TEXT_THICKNESS,
                   Config.LINE_TYPE)
        return image

class HandTracker:
    """手部追蹤器"""
    
    def __init__(self):
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            model_complexity=Config.MODEL_COMPLEXITY,
            min_detection_confidence=Config.MIN_DETECTION_CONFIDENCE,
            min_tracking_confidence=Config.MIN_TRACKING_CONFIDENCE,
            max_num_hands=2  # 允許檢測兩隻手
        )
        self.angle_calculator = AngleCalculator()
        self.gesture_recognizer = GestureRecognizer()
        self.effect_processor = EffectProcessor()
    
    def extract_landmarks(self, hand_landmarks) -> Tuple[List[Tuple[float, float]], List[int], List[int]]:
        """提取手部特徵點座標"""
        landmarks = []
        x_coords = []
        y_coords = []
        
        for landmark in hand_landmarks.landmark:
            x = landmark.x * Config.FRAME_WIDTH
            y = landmark.y * Config.FRAME_HEIGHT
            landmarks.append((x, y))
            x_coords.append(int(x))
            y_coords.append(int(y))
        
        return landmarks, x_coords, y_coords
    
    def get_bounding_box(self, x_coords: List[int], y_coords: List[int]) -> Tuple[int, int, int, int]:
        """獲取手部邊界框"""
        x_min = max(0, min(x_coords) - Config.MOSAIC_PADDING)
        y_min = max(0, min(y_coords) - Config.MOSAIC_PADDING)
        x_max = min(Config.FRAME_WIDTH, max(x_coords))
        y_max = min(Config.FRAME_HEIGHT, max(y_coords))
        
        return x_min, y_min, x_max, y_max
    
    def process_frame(self, frame: np.ndarray) -> np.ndarray:
        """處理單一幀圖像"""
        # 轉換顏色空間
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(rgb_frame)
        
        left_landmarks = None
        right_landmarks = None
        detected_gestures = []
        
        if results.multi_hand_landmarks and results.multi_handedness:
            for i, (hand_landmarks, handedness) in enumerate(zip(results.multi_hand_landmarks, results.multi_handedness)):
                # 提取特徵點
                landmarks, x_coords, y_coords = self.extract_landmarks(hand_landmarks)
                
                if landmarks:
                    # 判斷左手還是右手
                    is_left_hand = handedness.classification[0].label == 'Left'
                    
                    if is_left_hand:
                        left_landmarks = landmarks
                    else:
                        right_landmarks = landmarks
                    
                    # 計算手指角度
                    finger_angles = self.angle_calculator.calculate_finger_angles(landmarks)
                    
                    # 識別單手手勢
                    gesture = self.gesture_recognizer.recognize(finger_angles)
                    detected_gestures.append((gesture, x_coords, y_coords))
        
        # 檢測雙手愛心手勢
        heart_gesture = self.gesture_recognizer.recognize_dual_hand(left_landmarks, right_landmarks)
        
        if heart_gesture == HandGesture.HEART:
            # 計算雙手中心點
            if left_landmarks and right_landmarks:
                center_x = int((left_landmarks[9][0] + right_landmarks[9][0]) / 2)  # 中指根部平均
                center_y = int((left_landmarks[9][1] + right_landmarks[9][1]) / 2)
                frame = self.effect_processor.apply_heart_effect(frame, center_x, center_y)
                frame = self.effect_processor.draw_text(frame, "♥ LOVE ♥")
        else:
            # 處理單手手勢
            for gesture, x_coords, y_coords in detected_gestures:
                if gesture == HandGesture.NO:
                    x_min, y_min, x_max, y_max = self.get_bounding_box(x_coords, y_coords)
                    frame = self.effect_processor.apply_mosaic(frame, x_min, y_min, x_max, y_max)
                elif gesture != HandGesture.UNKNOWN:
                    frame = self.effect_processor.draw_text(frame, gesture.value)
        
        return frame
    
    def __del__(self):
        """清理資源"""
        if hasattr(self, 'hands'):
            self.hands.close()

class HandGestureApp:
    """手勢識別應用主類"""
    
    def __init__(self):
        self.cap = cv2.VideoCapture(0)
        self.hand_tracker = HandTracker()
        
        if not self.cap.isOpened():
            raise RuntimeError("無法開啟攝影機")
    
    def run(self):
        """運行應用程式"""
        print("手勢識別應用已啟動，按 'q' 鍵退出")
        print("新功能：雙手組成愛心可觸發愛心特效和粉色濾鏡！")
        
        try:
            while True:
                ret, frame = self.cap.read()
                if not ret:
                    print("無法讀取攝影機畫面")
                    break
                
                # 調整畫面大小
                frame = cv2.resize(frame, (Config.FRAME_WIDTH, Config.FRAME_HEIGHT))
                
                # 處理手勢識別
                processed_frame = self.hand_tracker.process_frame(frame)
                
                # 顯示結果
                cv2.imshow('Hand Gesture Recognition with Heart Effect', processed_frame)
                
                # 檢查退出條件
                if cv2.waitKey(5) & 0xFF == ord('q'):
                    break
                    
        except KeyboardInterrupt:
            print("程式被中斷")
        finally:
            self.cleanup()
    
    def cleanup(self):
        """清理資源"""
        self.cap.release()
        cv2.destroyAllWindows()

# 運行應用程式
if __name__ == "__main__":
    app = HandGestureApp()
    app.run()

手勢識別應用已啟動，按 'q' 鍵退出
新功能：雙手組成愛心可觸發愛心特效和粉色濾鏡！
